# Homework 2

## q1

In [ ]:
from fastembed import TextEmbedding

embedding_model = TextEmbedding('jinaai/jina-embeddings-v2-small-en')
documents = [
    "I just discovered the course. Can I join now?"
]

embeddings = list(embedding_model.embed(documents))

print(embeddings)


Fetching 5 files: 100%|██████████| 5/5 [00:03<00:00,  1.44it/s]

[array([-7.63946373e-02, -7.30555514e-02,  5.86501632e-02,  3.92670711e-02,
       -1.41796944e-02, -4.68342392e-02,  2.69862839e-02,  3.49039298e-02,
        1.53419599e-03, -1.10407681e-02,  5.57556382e-02, -2.88360335e-02,
       -4.87239656e-02, -9.90923940e-02,  6.02579685e-02, -1.86105496e-02,
       -8.81041160e-03, -3.01994898e-02, -1.42541591e-02, -3.87884289e-02,
       -2.76125646e-02, -6.94957634e-03,  3.20653388e-02, -5.17838445e-03,
        8.37960027e-02, -8.87329694e-02, -7.30260818e-02,  5.92846727e-02,
        4.07805582e-02,  7.72684693e-02, -4.60029423e-02,  3.98426848e-02,
        1.74821532e-02,  8.70262992e-03, -3.09291007e-02,  2.20470406e-02,
        4.76479838e-02,  1.76566216e-02, -3.62013813e-02, -3.53408182e-02,
       -5.92536405e-03,  2.97091266e-02,  8.05883488e-02,  1.07132627e-02,
       -5.38192486e-02, -9.62880078e-03, -1.17263736e-01,  3.36546383e-02,
        7.05763513e-03,  7.70160054e-03, -5.17482624e-02,  5.46955475e-02,
       -7.80289880e-02, 

In [7]:
min(embeddings[0])

np.float64(-0.11726373551188797)

## Q2

In [9]:
import numpy as np

def cosine_similarity(a, b):
    return np.dot(a, b) / (np.linalg.norm(a)) * np.linalg.norm(b)

In [11]:
doc = 'Can I still join the course after the start date?'
q2_doc_embedding = list(embedding_model.embed(doc))
print(cosine_similarity(embeddings[0], q2_doc_embedding[0]))

0.9008528856818036


## q3

In [12]:
documents = [{'text': "Yes, even if you don't register, you're still eligible to submit the homeworks.\nBe aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute.",
  'section': 'General course-related questions',
  'question': 'Course - Can I still join the course after the start date?',
  'course': 'data-engineering-zoomcamp'},
 {'text': 'Yes, we will keep all the materials after the course finishes, so you can follow the course at your own pace after it finishes.\nYou can also continue looking at the homeworks and continue preparing for the next cohort. I guess you can also start working on your final capstone project.',
  'section': 'General course-related questions',
  'question': 'Course - Can I follow the course after it finishes?',
  'course': 'data-engineering-zoomcamp'},
 {'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon’t forget to register in DataTalks.Club's Slack and join the channel.",
  'section': 'General course-related questions',
  'question': 'Course - When will the course start?',
  'course': 'data-engineering-zoomcamp'},
 {'text': 'You can start by installing and setting up all the dependencies and requirements:\nGoogle cloud account\nGoogle Cloud SDK\nPython 3 (installed with Anaconda)\nTerraform\nGit\nLook over the prerequisites and syllabus to see if you are comfortable with these subjects.',
  'section': 'General course-related questions',
  'question': 'Course - What can I do before the course starts?',
  'course': 'data-engineering-zoomcamp'},
 {'text': 'Star the repo! Share it with friends if you find it useful ❣️\nCreate a PR if you see you can improve the text or the structure of the repository.',
  'section': 'General course-related questions',
  'question': 'How can we contribute to the course?',
  'course': 'data-engineering-zoomcamp'}]

In [23]:
q3_embeddings = []
for document in documents:
    q3_embeddings.append(embedding_model.embed(document['question']))

cos_sims = []
for embedding in q3_embeddings:
    cos_sims.append(cosine_similarity(list(embedding), q2_doc_embedding[0]))

print(cos_sims.index(max(cos_sims)))
print(max(cos_sims))

0
[0.99568522]


## q4

In [25]:
q4_embeddings = []
for document in documents:
    full_text = document['question'] + ' ' + document['text']
    q4_embeddings.append(embedding_model.embed(full_text))

cos_sims = []
for embedding in q4_embeddings:
    cos_sims.append(cosine_similarity(list(embedding), q2_doc_embedding[0]))

print(cos_sims.index(max(cos_sims)))
print(max(cos_sims))

0
[0.92732519]


## q5

In [26]:
embedding_model = TextEmbedding('BAAI/bge-small-en')
print(len(list(embedding_model.embed(doc))[0]))

Fetching 5 files: 100%|██████████| 5/5 [00:03<00:00,  1.47it/s]

384


## q6

In [41]:
import requests
from qdrant_client import QdrantClient, models

client = QdrantClient("http://localhost:6333")

docs_url = 'https://github.com/alexeygrigorev/llm-rag-workshop/raw/main/notebooks/documents.json'
docs_response = requests.get(docs_url)
documents_raw = docs_response.json()

collection_name = "zoomcamp_faq"
# client.create_collection(
#     collection_name=collection_name,
#     vectors_config=models.VectorParams(
#         size=384,
#         distance=models.Distance.COSINE
#     )
# )

documents = []

points = []
id = 0
for course in documents_raw:
    course_name = course['course']
    if course_name != 'machine-learning-zoomcamp':
        continue

    for doc in course['documents']:
        doc['course'] = course_name
        documents.append(doc)
        text = doc['question'] + ' ' + doc['text']
        point = models.PointStruct(
            id=id,
            vector=models.Document(text=text, model='BAAI/bge-small-en'),
            payload={
                "text": doc['text'],
                "section": doc['section'],
                "course": course['course']
            }
        )
        points.append(point)
        id += 1
print(documents[0])

{'text': 'Machine Learning Zoomcamp FAQ\nThe purpose of this document is to capture frequently asked technical questions.\nWe did this for our data engineering course and it worked quite well. Check this document for inspiration on how to structure your questions and answers:\nData Engineering Zoomcamp FAQ\nIn the course GitHub repository there’s a link. Here it is: https://airtable.com/shryxwLd0COOEaqXo\nwork', 'section': 'General course-related questions', 'question': 'How do I sign up?', 'course': 'machine-learning-zoomcamp'}


In [42]:
client.upsert(
    collection_name=collection_name,
    points=points
)

UpdateResult(operation_id=1, status=<UpdateStatus.COMPLETED: 'completed'>)

In [43]:
def search(query, limit=1):
    results = client.query_points(
        collection_name=collection_name,
        query=models.Document(
            text=query,
            model='BAAI/bge-small-en'
        ),
        limit=limit,
        with_payload=True
    )
    return results

In [44]:
q = "I just discovered the course. Can I join now?"
search(q)

QueryResponse(points=[ScoredPoint(id=14, version=1, score=0.8703172, payload={'text': 'Yes, you can. You won’t be able to submit some of the homeworks, but you can still take part in the course.\nIn order to get a certificate, you need to submit 2 out of 3 course projects and review 3 peers’ Projects by the deadline. It means that if you join the course at the end of November and manage to work on two projects, you will still be eligible for a certificate.', 'section': 'General course-related questions', 'course': 'machine-learning-zoomcamp'}, vector=None, shard_key=None, order_value=None)])